### Face detection using webcam with NCS2 

Steps are following:
1. We need Intel Movidius NCS2 Accelerator and its correct installation.
2. Then, download model using "01_model_downloader.ipynb" named "face-detection-retail-0004" in FP16 Precision.
3. Note that here model comes from Intel Pre-trained models repo where models are already converted into IR formats with various quantization. 
4. FP16 Precision is a requirement for NCS2 to work on and check a list of supported layers.

-> Requirements of the face detection model:
* Originally, trained on Caffe but converted into IR format.
* Input size - (300, 300)
* Channel order - BGR 
* Output - [1, 1, N, 7] where N are the numbers of objects detected

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import cv2

#### Define parameters

In [3]:
# Parameters
MODEL_FPATH = "./models/intel/face-detection-retail-0004/FP16/face-detection-retail-0004.bin"
ARCH_FPATH = "./models/intel/face-detection-retail-0004/FP16/face-detection-retail-0004.xml"
CONF_THRESH = 0.5 # confidence of each object detected

#### Load model

In [4]:
# Load the model.
net = cv2.dnn.readNet(ARCH_FPATH, MODEL_FPATH)

# Specify NCS2 Myriad as a target device.
net.setPreferableBackend(cv2.dnn.DNN_BACKEND_INFERENCE_ENGINE)
net.setPreferableTarget(cv2.dnn.DNN_TARGET_MYRIAD) # DNN_TARGET_CPU (only if you don't have NCS2) or DNN_TARGET_MYRIAD

#### Initialize webcam video capture object

In [5]:
vid_cap = cv2.VideoCapture(0)
if not vid_cap.isOpened():
    raise IOError("Webcam cannot be opened!")

#### Read frame and inference on them (Press 'ESC' key to stop)

In [6]:
while True:
    # Capture frames
    ret, frame = vid_cap.read()
    
    # Prepare input blob and perform inference
    blob = cv2.dnn.blobFromImage(frame, size=(300, 300), ddepth=cv2.CV_8U)
    net.setInput(blob)
    out = net.forward()
    
    # Draw detected faces
    for detect in out.reshape(-1, 7):
        conf = float(detect[2])
        xmin = int(detect[3] * frame.shape[1])
        ymin = int(detect[4] * frame.shape[0])
        xmax = int(detect[5] * frame.shape[1])
        ymax = int(detect[6] * frame.shape[0])

        if conf > CONF_THRESH:
            cv2.rectangle(frame, (xmin, ymin), (xmax, ymax), color=(0, 255, 0), thickness=2)
    
    cv2.imshow('Input', frame)
    
    # Press "ESC" key to stop webcam
    if cv2.waitKey(1) == 27:
        break

# Release video capture object and close the window
vid_cap.release()
cv2.destroyAllWindows()
cv2.waitKey(1)

-1